In [1]:
import os
import pcse
import yaml
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
from multiprocessing import Pool
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from pcse.models import Wofost71_PP, Wofost71_WLP_FD
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse import fileinput as fi
from pcse.util import WOFOST71SiteDataProvider

In [2]:
agro_yaml = """
Version: 1.0
AgroManagement:
- {sim_start_date}:
    CropCalendar:
        crop_name: maize
        variety_name: Maize_VanHeemst_1988
        crop_start_type: sowing
        crop_start_date: {crop_start_date}
        crop_end_type: earliest
        crop_end_date: {crop_end_date}
        max_duration: 300
    TimedEvents: 
    -   event_signal: apply_n
        name:  Nitrogen application table
        comment: All nitrogen amounts in g N m-2
        events_table:
        - {fert_date}: {{amount: {n_rate}, recovery: 0.7}}
    StateEvents: null
"""

In [3]:
pts = gpd.read_file('../../CIAT/data/BEM_2019_Chiapas_Final_Clean.gpkg')
pts['SIM'] = ['CH{0:05d}'.format(i + 1) for i in range(len(pts))]
pts = pts.set_index('SIM')
pts['lon'] = pts.centroid.x
pts['lat'] = pts.centroid.y
pts['ref_date'] = pd.to_datetime(pts.fecha_de_siembra)
pts['nrate'] = pts.nitrogen/10


In [4]:
soildata = {'SMFCF': 0.2, 'SMW': 0.1, 'CRAIRC': 0.1, 'KSUB': 20.0,
            'RDMSOL': 200.0, 'K0': 100.0, 'SOPE': 10.0, 'SM0': 0.4}

cropdata = fi.YAMLCropDataProvider()
cropdata.set_active_crop('maize', 'Maize_VanHeemst_1988')

sitedata = WOFOST71SiteDataProvider(WAV=100, CO2=360)

parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

In [11]:
def run(site):
        wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{site}_adv010.csv'
        wdp = fi.CSVWeatherDataProvider(wfile)

        # get new start date
        crop_start_date = pts.ref_date.dt.date.loc[site]
        sim_start_date = crop_start_date - dt.timedelta(days = 90)
        crop_end_date = crop_start_date + dt.timedelta(days = 210)
        fert_date = crop_start_date + dt.timedelta(days = 30)
        n_rate = 1

        # update agromanagement with new start date and load it with yaml.load
        tmp = agro_yaml.format(sim_start_date = sim_start_date,
                               crop_start_date = crop_start_date,
                               crop_end_date = crop_end_date,
                               fert_date = fert_date,
                               n_rate = n_rate)
        agromanagement = yaml.load(tmp, yaml.FullLoader)

        wofsim = Wofost71_WLP_FD(parameters, wdp, agromanagement)
        wofsim.run_till_terminate()
        output = wofsim.get_output()
        df = pd.DataFrame(output).set_index("day")
        df['SIM'] = site
        return(df)


In [9]:
# site = (sites[0])
# for site in tqdm(pts.index):
#     wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{site}_adv000.csv'
#     w = pd.read_csv(wfile, skiprows = 13)
#     w.VAP = np.clip(w.VAP, 0.06, 199.3)
    
#     save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{site}_adv000f.csv'
#     with open(save_file, 'w') as sf:
#         with open(wfile) as f:
#             for r in range(14):
#                 sf.writelines(f.readline())
    
#     w.DAY = w.DAY.astype('str')
#     w.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)


In [12]:
sites = pts.index.values
output = []
if __name__ == '__main__':
    with Pool(processes=30) as p:
        with tqdm(total=len(pts)) as pbar:
            for i, df in enumerate(p.imap_unordered(run, sites)):
                output.append(df)
                pbar.update()

output_df = pd.DataFrame([o.iloc[-1] for o in output])
output_df.to_hdf('../data/PSCE_npadv010.h5', key = 'SIM')

--- Logging error ---
Traceback (most recent call last):
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/logging/handlers.py", line 70, in emit
    self.doRollover()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/logging/handlers.py", line 167, in doRollover
    os.rename(sfn, dfn)
FileNotFoundError: [Errno 2] No such file or directory: '/home/rodrigo7/.pcse/logs/pcse.log.6' -> '/home/rodrigo7/.pcse/logs/pcse.log.7'
Call stack:
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-pa

FileNotFoundError: [Errno 2] No such file or directory: '/home/rodrigo7/.pcse/logs/pcse.log.7'
Traceback (most recent call last):
FileNotFoundError: [Errno 2] No such file or directory: '/home/rodrigo7/.pcse/logs/pcse.log.6' -> '/home/rodrigo7/.pcse/logs/pcse.log.7'
Traceback (most recent call last):
Call stack:
Call stack:
FileNotFoundError: [Errno 2] No such file or directory: '/home/rodrigo7/.pcse/logs/pcse.log.3' -> '/home/rodrigo7/.pcse/logs/pcse.log.4'
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/logging/handlers.py", line 70, in emit
    self.doRollover()
Call stack:
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/logging/handlers.py", line 166, in doRollover
    os.remove(dfn)
Call stack:
FileNotFoundError: [Errno 2] No such file or directory: '/home/rodrigo7/.pcse/logs/pcse.log.7'
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/logging/handlers.py", line 70, in emit
    self.doRollover()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/runpy.py", line 193, in _run_module

  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/gen.py", line 787, in inner
    self.run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/asyncio/base_events.py", line 1782, in _run_once
    handle._run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/asyncio/base_events.py", line 1782, in _run_once
    handle._run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/ioloop.py", line 690, in <lambda>
 

  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 268, in dispatch_shell
    yield gen.maybe_future(handler(stream, idents, msg))
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/gen.py", line 209, in wrapper
    yielded = next(result)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/gen.py", line 209, in wrapper
    yielded = next(result)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/gen.py", line 209, in wrapper
    yielded = next(result)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/tornado/gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/opt/anaconda3/envs/GEOA

  File "<ipython-input-12-739ca1795a84>", line 4, in <module>
    with Pool(processes=30) as p:
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/multiprocessing/process.py", line 112, in start
    self._popen = self._Popen(self)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/ipykernel/zmqshell.py", line 536, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 541, in execute_request
    user_expressions, allow_stdin,
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2858, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/multiprocessing/context.py", line 277, in _Popen
    return Popen(proc

  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 208, in _run
    self.agromanager(self.day, self.drv)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/multiprocessing/pool.py", line 176, in __init__
    self._repopulate_pool()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3254, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/multiprocessing/process.py", line 112, in start
    self._popen = self._Popen(self)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/multiprocessing/popen_fork.py", line 74, in _launch
    code = process_obj._bootstrap()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/agromanager.py", line 916, in __call__
 

  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 228, in run_till_terminate
    self._run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 228, in run_till_terminate
    self._run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 208, in _run
    self.agromanager(self.day, self.drv)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/agromanager.py", line 916, in __call__
    self.crop_calendars[0](day)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 208, in _run
    self.agromanager(self.day, self.drv)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/engine.py", line 228, in run_till_terminate
    self._run()
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcse/agromanager.py", line 163, in __call__
    self.logger.info(msg)
  File "/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pcs